In [2]:
# %% import libaries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing, model_selection
from keras.models import Sequential, load_model
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

In [3]:
# %% Preprocess

In [4]:
# import data
data = pd.read_csv('drug200.csv')

In [5]:
# Nulls?
data.isnull().sum()

Age            0
Sex            0
BP             0
Cholesterol    0
Na_to_K        0
Drug           0
dtype: int64

In [6]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1]

In [7]:
# Encode feature(s)?
data.info()

Sex_encoder = LabelEncoder()
Sex_encoder.fit(X["Sex"])
X["Sex"] = Sex_encoder.transform(X["Sex"])

BP_encoder = LabelEncoder()
BP_encoder.fit(X["BP"])
X["BP"] = BP_encoder.transform(X["BP"])

Cholesterol_encoder = LabelEncoder()
Cholesterol_encoder.fit(X["Cholesterol"])
X["Cholesterol"] = Cholesterol_encoder.transform(X["Cholesterol"])

Y_encoder = LabelEncoder()
Y_encoder.fit(Y)
Y = Y_encoder.transform(Y)
Y = np_utils.to_categorical(Y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Age          200 non-null    int64  
 1   Sex          200 non-null    object 
 2   BP           200 non-null    object 
 3   Cholesterol  200 non-null    object 
 4   Na_to_K      200 non-null    float64
 5   Drug         200 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 9.5+ KB


In [8]:
# split data
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y,test_size = 0.2, random_state = 0)

In [9]:
# %% NN architecture

#input_dim = train_x.shape[1]

model = Sequential()
model.add(Dense(8, input_dim = 5 , activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'] )

Metal device set to: Apple M1 Pro


2022-01-19 19:17:00.763435: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-19 19:17:00.763560: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
model.fit(train_x, train_y, epochs = 10, batch_size = 2)

Epoch 1/10


2022-01-19 19:17:00.849082: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-19 19:17:00.995161: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


80/80 [==============================] - 1s 8ms/step - loss: 3.2659 - accuracy: 0.5250
Epoch 2/10
80/80 [==============================] - 1s 8ms/step - loss: 1.6044 - accuracy: 0.5813
Epoch 3/10
80/80 [==============================] - 1s 8ms/step - loss: 1.2023 - accuracy: 0.5375
Epoch 4/10
80/80 [==============================] - 1s 8ms/step - loss: 1.1038 - accuracy: 0.5500
Epoch 5/10
80/80 [==============================] - 1s 7ms/step - loss: 1.0981 - accuracy: 0.5688
Epoch 6/10
80/80 [==============================] - 1s 8ms/step - loss: 1.0531 - accuracy: 0.5563
Epoch 7/10
80/80 [==============================] - 1s 8ms/step - loss: 1.0286 - accuracy: 0.5688
Epoch 8/10
80/80 [==============================] - 1s 8ms/step - loss: 1.0137 - accuracy: 0.5688
Epoch 9/10
80/80 [==============================] - 1s 7ms/step - loss: 1.0172 - accuracy: 0.5875
Epoch 10/10
80/80 [==============================] - 1s 8ms/step - loss: 1.0026 - accuracy: 0.5688


In [11]:
model.summary() # model architecture

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 48        
                                                                 
 dense_1 (Dense)             (None, 10)                90        
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 10)                110       
                                                                 
 dense_4 (Dense)             (None, 5)                 55        
                                                                 
Total params: 413
Trainable params: 413
Non-trainable params: 0
_________________________________________________________________


In [12]:
checkpointer = ModelCheckpoint(filepath="weights.hdf5", monitor = 'val_acc', verbose=1, save_best_only=True) # save model's best weights

In [15]:
history = model.fit(
                    train_x,
                    train_y,
                    epochs=30,
                    batch_size=30,
                    callbacks = [checkpointer],
                    validation_data = (test_x, test_y)
                    )

Epoch 1/30
6/6 [==============================] - 0s 17ms/step - loss: 0.9166 - accuracy: 0.6188 - val_loss: 0.9254 - val_accuracy: 0.6500
Epoch 2/30
6/6 [==============================] - 0s 13ms/step - loss: 0.9158 - accuracy: 0.6188 - val_loss: 0.9264 - val_accuracy: 0.6500
Epoch 3/30
6/6 [==============================] - 0s 14ms/step - loss: 0.9156 - accuracy: 0.6125 - val_loss: 0.9274 - val_accuracy: 0.6500
Epoch 4/30
6/6 [==============================] - 0s 14ms/step - loss: 0.9141 - accuracy: 0.6125 - val_loss: 0.9274 - val_accuracy: 0.6500
Epoch 5/30
6/6 [==============================] - 0s 14ms/step - loss: 0.9130 - accuracy: 0.6125 - val_loss: 0.9264 - val_accuracy: 0.6500
Epoch 6/30
6/6 [==============================] - 0s 14ms/step - loss: 0.9153 - accuracy: 0.6188 - val_loss: 0.9252 - val_accuracy: 0.6500
Epoch 7/30
6/6 [==============================] - 0s 15ms/step - loss: 0.9118 - accuracy: 0.6188 - val_loss: 0.9281 - val_accuracy: 0.6500
Epoch 8/30
6/6 [===========

In [ ]:
model_name = "drug.h5"
model.save(model_name)
model = load_model(model_name)

In [16]:
# %% Predictions

y_pred = model.predict(test_x)

2022-01-19 19:17:29.844633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred = Y_encoder.inverse_transform(y_pred)

In [ ]:
Y = np.argmax(Y, axis = 1)
Y = Y_encoder.inverse_transform(Y)

In [ ]:
test_y = np.argmax(test_y, axis = 1)
test_y = Y_encoder.inverse_transform(test_y)

In [ ]:
cm = confusion_matrix(y_pred, test_y)
cm

In [ ]:
print("\nHistory Keys:\n")
print(history.history.keys())

In [ ]:
plt.subplots() # open a new plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
plt.subplots() # open a new plot
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'])
plt.show()